<pre>
      _                 _              _____ 
     | |               | |            / __  \
  ___| |__   __ _ _ __ | |_ ___ _ __  `' / /'
 / __| '_ \ / _` | '_ \| __/ _ \ '__|   / /  
| (__| | | | (_| | |_) | ||  __/ |    ./ /___
 \___|_| |_|\__,_| .__/ \__\___|_|    \_____/
                 | |                         
                 |_|                         
</pre>

## Глава 2

- [2 Запросы SQL к связанным таблицам](#2-Запросы-SQL-к-связанным-таблицам)
    - [2.1 Связи между таблицами](#2.1-Связи-между-таблицами)
        - [Связь «один ко многим»](#Связь-«один-ко-многим»)
        - [Связь «многие ко многим»](#Связь-«многие-ко-многим»)
        - [Создание и заполнение таблицы author](#Создание-и-заполнение-таблицы-author)
        - [Добавление в таблицу внешнего ключа](#Добавление-в-таблицу-внешнего-ключа)
        - [Действия-при-удалении-записи-главной-таблицы](#Действия-при-удалении-записи-главной-таблицы)
    - [2.2 Запросы на выборку, соединение таблиц](#2.2-Запросы-на-выборку,-соединение-таблиц)
    - [2.3 Запросы корректировки, соединение таблиц](#2.3-Запросы-корректировки,-соединение-таблиц)
    - [](#)
    - [](#)
    - [](#)



#  2 Запросы SQL к связанным таблицам
[`⌂`](#Глава-2)


<hr>

## 2.1 Связи между таблицами

[`⌂`](#Глава-2)

<hr>

### Связь «один ко многим»

[`⌂`](#Глава-2)



Рассмотрим таблицу **book**

In [1]:
from  utils import sql, schema, show, table, load_csv, load_sql
import pandas as pd

In [2]:
# ограничиваем вывод 8 строками
# если будет больше 8 строк,
# то отобразятся первые 4 строки и последние 4 строки
pd.options.display.max_rows = 8

In [3]:
load_sql('sql/chapter2/book.sql')
load_csv('data/chapter2.0/book.csv')

CREATE TABLE book (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title VARCHAR(50) NOT NULL,
    author VARCHAR(30) NOT NULL,
    price DECIMAL(8, 3) NOT NULL,
    amount INTEGER NOT NULL,
    
    UNIQUE(title, author) ON CONFLICT IGNORE
)

,title,author,price,amount
0,Бесы,Достоевский Ф.М.,800,3
1,Игрок,Достоевский Ф.М.,700,8
2,Война и мир,Толстой Л.Н.,1000,3
3,Анна Каренина,Толстой Л.Н.,1200,7
...,...,...,...,...
17,Братья Карамазовы,Достоевский Ф.М.,888,9
18,Преступление и наказание,Достоевский Ф.М.,452,7
19,Обломов,Гончаров И.А.,365,3
20,Стихотворения и поэма,Есенин С.А.,578,6


В этой таблице фамилии авторов повторяются для нескольких книг. А что, если придется вместо инициалов для каждого автора хранить его полное имя и отчество? Тогда, если в таблице содержится информация о 50 книгах Достоевского, придется 50 раз исправлять «Ф.М.» на «Федор Михайлович». При этом, если в некоторых записях использовать «Фёдор Михайлович» (c буквой ё), то мы вообще получим двух разных авторов... 

Чтобы устранить эту проблему в реляционных базах данных создается новая таблица **author**,  в которой перечисляются все различные авторы, а затем эта таблица связывается с таблицей **book**. При этом такая связь называется «**один ко многим**», таблица **author** называется ``главной``, таблица **book** – связанной или ``подчиненной``.

Связь «**один ко многим**» имеет место, когда одной записи главной таблицы соответствует несколько записей связанной таблицы, а каждой записи связанной таблицы соответствует только одна запись главной таблицы. Обозначается это так:

<img src="img/2.1/many_to_one.svg">

**Этапы реализации связи «один ко многим»** на следующем примере:

Один автор может написать несколько книг, а одна книга написана только одним автором. Для каждой книги известны ее количество и цена.

1. Создать таблицу **author**,  в которую включить уникальных авторов книг, хранящихся на складе:

<img src="img/2.1/table_1.svg">

2. Обе таблицы должны содержать первичный ключ, в таблице  **book** он уже есть,  в таблицу **author** добавим ключ **author_id**:

<img src="img/2.1/table_2.svg">

3. Включить в таблицу book связанный столбец (внешний ключ, **FOREIGN KEY**), соответствующий по имени и типу ключевому столбцу главной таблицы (в нашем случае это столбец **author_id**). Для наглядности связь на схеме обозначается стрелкой от ключевого столбца главной таблицы к внешнему ключу связной таблицы:

<img src="img/2.1/table_3.svg">

<hr>

### Связь «многие ко многим»

[`⌂`](#Глава-2)

[На предыдущем шаге](#Связь-«один-ко-многим») мы реализовали связь «**один ко многим**» для книг и авторов. Она означает, что каждый автор написал несколько книг, но каждую книгу написал только один автор. На самом деле, это не совсем верное утверждение. Например, книга «12 стульев» написана двумя авторами Ильфом И.А. и Петровым Е.П. С другой стороны, эти авторы написали и другие книги, например «Золотой теленок».

Для соединения таких таблиц используется связь «**многие ко многим**».

Связь «**многие ко многим**» имеет место когда каждой записи одной таблицы соответствует несколько записей во второй, и наоборот, каждой записи второй таблицы соответствует несколько записей в первой. Обозначается это так:

<img src="img/2.1/many_to_many.svg">

**Этапы реализации связи «многие ко многим»**  на следующем примере:

Один автор может написать несколько книг, а одна книга может быть написана несколькими авторами. Для каждой книги известны ее количество и цена.

1. Создать таблицу **author**,  в которую включить уникальных авторов книг, хранящихся на складе:

<img src="img/2.1/mtm_table_1.svg">

2. В обеих таблицах необходимо определить первичный ключ, в нашем случае в таблице **book** он уже есть, поэтому достаточно включить первичный ключ **author_id** в таблицу **author**:

<img src="img/2.1/mtm_table_2.svg">

3. Создать новую таблицу-связку, состоящую из двух столбцов, соответствующих по имени и типу ключевым столбцам исходных таблиц. Каждый из этих столбцов является внешним ключом (FOREIGN KEY)  и связан с ключевым столбцом каждой таблицы. Для наглядности связи на схеме обозначаются стрелкой от ключевого столбца исходной таблицы к внешнему ключу связной таблицы

<img src="img/2.1/mtm_table_3.svg">

4. Дальше необходимо определиться с первичным ключом таблицы-связки. Можно сделать два ключевых столбца, тогда все записи в этой таблице должны быть уникальными, то есть не повторяться. Для связи автор-книга этот вариант подходит. Но в некоторых случаях записи в таблице-связке могут повторяться, например, если мы будем продавать книги покупателям (один человек может купить несколько книг, а одну и ту же книгу могут купить несколько человек). Тогда в таблицу-связку включают дополнительные столбцы для идентификации записей, например, дату продажи,  также в таблицу-связку добавляют первичный ключ. Мы воспользуемся вторым способом:

<img src="img/2.1/mtm_table_4.svg">




<hr>

### Создание и заполнение таблицы author

[`⌂`](#Глава-2)

Создадим таблицу **author**


In [4]:
load_sql('sql/chapter2/author.sql')

CREATE TABLE author (
    author_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name_author VARCHAR(50)
)

И вставим всех авторов из таблицы book


In [5]:
sql("""
INSERT INTO author(name_author)
SELECT DISTINCT author 
FROM book
""")
table('author')

INSERT INTO author(name_author)
SELECT DISTINCT author 
FROM book

,author_id,name_author
0,1,Толстой Л.Н.
1,2,Булгаков М.А.
2,3,Достоевский Ф.М.
3,4,Чехов А.П.
...,...,...
5,6,Гоголь Н.В.
6,7,Гончаров И.А.
7,8,Тургенев И.С.
8,9,Есенин С.А.


<hr>

### Добавление в таблицу внешнего ключа

[`⌂`](#Глава-2)

При создании зависимой таблицы (таблицы, которая содержит внешние ключи) необходимо учитывать, что :

- каждый внешний ключ должен иметь такой же тип данных, как связанное поле главной таблицы (в наших примерах это INT);
- необходимо указать главную для нее таблицу и столбец, по которому осуществляется связь:

```SQLite
FOREIGN KEY (связанное_поле_зависимой_таблицы)  
REFERENCES главная_таблица (связанное_поле_главной_таблицы)
```

По умолчанию любой столбец, кроме ключевого, может содержать значение `NULL`. При создании таблицы это можно переопределить,  используя  ограничение `NOT NULL` для этого столбца:

```SQLite
CREATE TABLE таблица (
    столбец_1 INT NOT NULL, 
    столбец_2 VARCHAR(10) 
);
```

В созданной таблице в столбец_1 не может содержать пустое значение, а столбец_2 - может.

Для внешних ключей рекомендуется устанавливать ограничение `NOT NULL` (если это совместимо с другими опциями, которые будут рассмотрены в следующем шаге).

Изменим таблицу **book** добавив в неё внешний ключ на таблицу **author** 



In [ ]:
sql("""
ALTER TABLE book
ADD COLUMN author_id;
""")

In [ ]:
sql("""
UPDATE
    book
SET
    author_id = (SELECT author_id FROM author WHERE author.name_author = book.author)
""")
table('book')

In [ ]:
sql("""
ALTER TABLE book
ADD CONSTRAINT fk_author
    FOREIGN KEY (author_id) 
    REFERENCES author(author_id);
""")
https://stackoverflow.com/questions/1884818/how-do-i-add-a-foreign-key-to-an-existing-sqlite-tablehttps://stackoverflow.com/questions/1884818/how-do-i-add-a-foreign-key-to-an-existing-sqlite-table

<hr>

### Действия при удалении записи главной таблицы

[`⌂`](#Глава-2)

<hr>

## 2.2 Запросы на выборку, соединение таблиц

[`⌂`](#Глава-2)

<hr>

## 2.3 Запросы корректировки, соединение таблиц

[`⌂`](#Глава-2)

<hr>

## Выборка данных по условию

[`⌂`](#Глава-2)